# Initial Setup

- We will be importing the necessary dependencies and importing the dataset csv into a dataframe.
- The first column will be dropped as it serves no purpose other than denote the row number
- The column names will be standardized to have lower case letters separated by underscore(_)
- The target value we are trying to predict with this exercise is Estimated Shares Outstanding (ESO) therefore we will be taking it separately into a target set.

In [ ]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor


import seaborn as sns

# increase number of values displayed by query
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# read data from csv
df = pd.read_csv('fundamentals.csv')

# To check if the data is loaded correctly
print(len(df))
# print(df.info())

# drop the first column since it only has row count
df = df.drop(df.columns[0], axis=1)

# check if column was dropped
# print(df.info())

# Create a standard list of column names
standardized_column_names = [
    'ticker_symbol',
    'period_ending',
    'accounts_payable',
    'accounts_receivable',
    'additional_income_expense_items',
    'after_tax_roe',
    'capital_expenditures',
    'capital_surplus',
    'cash_ratio',
    'cash_and_cash_equivalents',
    'changes_in_inventories',
    'common_stocks',
    'cost_of_revenue',
    'current_ratio',
    'deferred_asset_charges',
    'deferred_liability_charges',
    'depreciation',
    'earnings_before_interest_and_tax',
    'earnings_before_tax',
    'effect_of_exchange_rate',
    'equity_earnings_loss_unconsolidated_subsidiary',
    'fixed_assets',
    'goodwill',
    'gross_margin',
    'gross_profit',
    'income_tax',
    'intangible_assets',
    'interest_expense',
    'inventory',
    'investments',
    'liabilities',
    'long_term_debt',
    'long_term_investments',
    'minority_interest',
    'misc_stocks',
    'net_borrowings',
    'net_cash_flow',
    'net_cash_flow_operating',
    'net_cash_flows_financing',
    'net_cash_flows_investing',
    'net_income',
    'net_income_adjustments',
    'net_income_applicable_to_common_shareholders',
    'net_income_cont_operations',
    'net_receivables',
    'non_recurring_items',
    'operating_income',
    'operating_margin',
    'other_assets',
    'other_current_assets',
    'other_current_liabilities',
    'other_equity',
    'other_financing_activities',
    'other_investing_activities',
    'other_liabilities',
    'other_operating_activities',
    'other_operating_items',
    'pre_tax_margin',
    'pre_tax_roe',
    'profit_margin',
    'quick_ratio',
    'research_and_development',
    'retained_earnings',
    'sale_and_purchase_of_stock',
    'sales_general_and_admin',
    'short_term_debt_current_portion_of_long_term_debt',
    'short_term_investments',
    "total_assets",
    "total_current_assets",
    "total_current_liabilities",
    "total_equity",
    "total_liabilities",
    "total_liabilities_&_equity",
    "total_revenue",
    "treasury_stock",
    "for_year",
    "earnings_per_share",
    "estimated_shares_outstanding"
]

print(len(standardized_column_names), "column_names")

# rename columns
df.columns = standardized_column_names

# verify if columns were renamed
# print(df.info())

original_df = df.copy()

# create a dataframe with the target columns (estimated_shares_outstanding)
target_df = df[['estimated_shares_outstanding']].copy()
# verify if target_df has estimated_shares_outstanding values
print(target_df.info())

# drop estimated_shares_outstanding column from the original dataframe
df = df.drop(['estimated_shares_outstanding'], axis=1)


# verify if estimated_shares_outstanding was dropped
# print(df.info())


# Handling missing values

In [ ]:
print(df.info())

# check for null values in ascending order
df.isnull().sum().sort_values(ascending=False)

According to the above the empty column count are as follows
- quick_ratio                                          299
- cash_ratio                                           299
- current_ratio                                        299
- earnings_per_share                                   219
- for_year                                             173


### Handling Ratios

The equations for the ratios are as follows

Current Ratio
> Current Ratio = Total Current Assets / Total Current Liabilities


Quick Ratio
> Quick Ratio = (Total Current Assets - Inventory) / Total Current Liabilities

Cash Ratio
> Cash Ratio = Cash and Cash Equivalents / Total Current Liabilities

*In all of the above equations we see liabilities as a denominator. My hypothesis is that the ratios are empty because Total Current liabilities are 0 (anything defined by zero is undefined, empty in this case)*

In [ ]:
# Testing above hypothesis if the null values are due to the fact that the current liabilities are zero

# number of rows where total current liabilities are zero and current ratio is null
tcl_zero_cr_null = df[(df['total_current_liabilities'] == 0) & (df['current_ratio'].isnull())]
print(len(tcl_zero_cr_null))

# number of rows where total current liabilities are zero and quick ratio is not null
tcl_zero_qr_null = df[(df['total_current_liabilities'] == 0) & (df['quick_ratio'].isnull())]
print(len(tcl_zero_qr_null))

# number of rows where total current liabilities are zero and cash ratio is not null
tcl_zero_casr_null = df[(df['total_current_liabilities'] == 0) & (df['cash_ratio'].isnull())]
print(len(tcl_zero_casr_null))

The above code proves the hypothesis where the number of ratios being empty and the total current liabilities = 0 is the same.

When handling missing values in a dataset, a common approach is to impute missing values. However, in this use case we cannot impute them as the denominator is zero and putting any other value derived from mean, median or mode would not be accurate.

Additionally these three columns related to ratios are derived columns, therefore we can drop them.

In [ ]:
# Drop Cash Ratio, Current Ratio and Quick Ratio
df.drop(['current_ratio', 'quick_ratio' ,'cash_ratio'], axis=1, inplace=True)

print(df.info())

### Handling missing 'earnings_per_share'

Earnings per share can be calculated with the following equation
> Earnings Per Share = Net Income / Estimated Shares Outstanding

In [ ]:
# Verify if earnings_per_share = net_income / estimated_shares_outstanding
eso_verification_merged_df = pd.merge(df[['net_income', 'earnings_per_share']], target_df, left_index=True, right_index=True)

# drop rows where earnings_per_share is null
eso_verification_merged_df = eso_verification_merged_df[eso_verification_merged_df['earnings_per_share'].notnull()]

eso_verification_merged_df['calculated_eps'] = eso_verification_merged_df['net_income'] / eso_verification_merged_df['estimated_shares_outstanding']

# compare two float64 columns with tolerance epsilon
eso_verification_merged_df['eso_verification'] = np.isclose(eso_verification_merged_df['calculated_eps'], eso_verification_merged_df['earnings_per_share'], atol=1e-10)

# get rows where eso_verification is False
failed_eso_verification = eso_verification_merged_df[eso_verification_merged_df['eso_verification'] == False]

print(len(failed_eso_verification))

The above code proves that the equation to find Earnings per Share from Estimated Shares Outstanding is valid as the length of failed_eso_verification is 0.

As Earnings per share is a derived column and is something that can be derived from the target column, we can drop the entire column as part of handling missing value.

In [ ]:
# Drop earnings_per_share column from data frame
df.drop(['earnings_per_share'], axis=1, inplace=True)

### Handling missing 'for_year'

The 'for_year' refers the the financial year to which the the row belongs to. Financial years don't always follow the start and end of the gregorian calendar we normally use. For example the financial year 2022 might end on the 1st of April 2023.

We cannot simply imput the year value from 'period_ending' for the financial year since it would be accurate. Instead we can drop the 'for_year' column entirely as the relevant inference can be made with the 'period_ending' alone

In [ ]:
# drop for_year column
df.drop(['for_year'], axis=1, inplace=True)

In [ ]:
# check for null values in ascending order
df.isnull().sum().sort_values(ascending=False)

# Handle outliers - TODO

In [ ]:
# TODO

# Generate Q-Q plot and Histograms

In [ ]:


for feature in df.columns:
    if not pd.api.types.is_numeric_dtype(df[feature]):
        continue

    # Get the feature data
    feature_data = df[feature]

    # transform the feature
    transformed_feature = np.log1p(feature_data)

    # Create a Q-Q plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].set_title(f'Q-Q Plot of {feature}')
    stats.probplot(feature_data, dist='norm', plot=axes[0])

    # Create a histogram
    axes[1].set_title(f'Histogram: {feature}')
    sns.histplot(transformed_feature, kde=True, ax=axes[1])

    # Show the plots
    plt.show()

    

# Apply feature  coding

In [ ]:
# Apply one-hot encoding to ticker_symbol column
encoder = OneHotEncoder()

# Fit and transform the 'ticker_symbol' column using OneHotEncoder
ticker_encoded = encoder.fit_transform(df[['ticker_symbol']])

# Convert the encoded result into a DataFrame
ticker_encoded_df = pd.DataFrame(ticker_encoded.toarray(), columns=encoder.get_feature_names_out(['ticker_symbol']))

# Concatenate the encoded DataFrame with the original DataFrame
df_one_hot = pd.concat([df, ticker_encoded_df], axis=1)

# Drop the original 'ticker_symbol' column
df_one_hot.drop('ticker_symbol', axis=1, inplace=True)

print(ticker_encoded_df.columns)
print(df_one_hot.columns)


# Scale and Standardize features

In [ ]:
scaler = MinMaxScaler()

# get numeric columns from df
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# apply min-max scaling to numeric columns in df
df_scaled = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)






for feature in df_scaled.columns:
    if not pd.api.types.is_numeric_dtype(df_scaled[feature]):
        continue

    # Get the feature data
    feature_data = df_scaled[feature]

    # transform the feature
    transformed_feature = np.log1p(feature_data)

    # Create a Q-Q plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].set_title(f'Q-Q Plot of {feature}')
    stats.probplot(feature_data, dist='norm', plot=axes[0])

    # Create a histogram
    axes[1].set_title(f'Histogram: {feature}')
    sns.histplot(transformed_feature, kde=True, ax=axes[1])

    # Show the plots
    plt.show()


# Feature Descretization - TODO

# Feature Reduction with PCA OR SVD - TODO

# Identification of significant and independent features - TODO

# Prediction

## Linear regression with cross validation - TODO

## Lasso regression with cross validation - TODO

## Ridge regression with cross validation - TODO

# Comparison of different regression models - TODO